# Hybrid Recommender System

## Option 1

Create three hybrid recommender systems by combining a collaborative filtering model with a content-based model using the following three
strategies:

In [ ]:
import pickle
from collections import defaultdict
import pandas as pd
import numpy as np

with open('prediction_SVD.pickle', 'rb') as file:
    prediction_SVD = pickle.load(file)

with open('prediction_TFIDF.pickle', 'rb') as file:
    prediction_TFIDF = pickle.load(file)

train_df = pd.read_pickle("train_dataframe.pkl") 
test_df = pd.read_pickle("test_dataframe.pkl")

### Parallel combination strategy

that re-ranks the items by combining
the individual rankings from the two models with some aggregation
function such as the sum, average, minimum or maximum.

In [ ]:
pc_pred = {}
for user_id, items_svd in prediction_SVD.items():
    if user_id in prediction_TFIDF:
        items_tfidf = prediction_TFIDF[user_id]
        pc_pred[user_id] = {}

        for item, rating_svd in items_svd.items():
            if item in items_tfidf:
                rating_tfidf = items_tfidf[item]
                avg_rating = (rating_svd + rating_tfidf) / 2
                pc_pred[user_id][item] = avg_rating
pc_pred

In [ ]:
from collections import defaultdict
from typing import Dict, Any, List
def get_top_k_for_all_users(predictions: Dict[str, Dict[str, float]], k: int) -> defaultdict:
   
    top_k_recommendations = defaultdict(list)

    for user_id, user_predictions in predictions.items():
        sorted_items = sorted(user_predictions.items(), key=lambda x: x[1], reverse=True)
        
        top_k_items = [(item_id, round(rating, 3)) for item_id, rating in sorted_items[:k]]
        
        top_k_recommendations[user_id] = top_k_items

    return top_k_recommendations

top10_parallel = get_top_k_for_all_users(pc_pred, 10)

### Switching strategy

that uses the recommendations from the collabo-
rative filtering model for some users and the recommendations from
the content-based model for other users chosen by a predefined con-
dition.

In [ ]:
# condition: user interaction frequency
user_interaction_count = train_df.groupby('user_id').size().reset_index(name='interaction_count')
sparsity_threshold = 10

In [ ]:
sw_pred = {}
for user_id, items_svd in prediction_SVD.items():
    sw_pred[user_id] = {}

    if user_interaction_count.get(user_id, 0) >= sparsity_threshold:
        for item, rating_svd in items_svd.items():
            sw_pred[user_id][item] = rating_svd
            
    else:
        if user_id in prediction_TFIDF:
            for item, rating_tfidf in prediction_TFIDF[user_id].items():
                sw_pred[user_id][item] = rating_tfidf
sw_pred

In [ ]:
top10_switch = get_top_k_for_all_users(sw_pred, 10)

### Pipelining (sequential) strategy

where a level of one model is used as
input to the other model

In [ ]:
#first content based - get >= 3, then collaborative filtering
pp_pred = {}

for user_id, items_tfidf in prediction_TFIDF.items():
    pp_pred[user_id] = {}

    for item, rating_tfidf in items_tfidf.items():
        if rating_tfidf >= 3:
            if user_id in prediction_SVD and item in prediction_SVD[user_id]:
                pp_pred[user_id][item] = prediction_SVD[user_id][item]
pp_pred

In [ ]:
top10_pipeline = get_top_k_for_all_users(pp_pred, 10)

In [ ]:
import numpy as np
from __future__ import (absolute_import, division, print_function, unicode_literals)
from collections import defaultdict
from surprise import Dataset

test_df['new_label'] = test_df['rating'].apply(lambda x: 1 if x >= 3 else 0)
test_df[test_df['new_label'] == 1]

def precision_at_k(top_k: Dict[str, List[str]], df_test: pd.DataFrame, k: int) -> Dict[str, float]:
    """Compute precision at k for each user
    Args:
        top_k: A dictionary where keys are user ids (str) and values are lists of (item_id, rating_estimation) tuples.
        df_test: Pandas DataFrame containing user-item ratings in the test split.
        k: The number of recommendations to output for each user.
    Returns:
        A dictionary where keys are user ids (str) and values are P@k (float) for each user.
    """
    
    precisions = defaultdict(float)
    
    # Only consider relevant items (rating ≥ 4.0)
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()
    
    for user, recommended_items in top_k.items():
        recommended_set = {item for item, _ in recommended_items[:k]}  # Take top-k items
        
        if user in relevant_items:
            num_relevant_at_k = len(recommended_set & relevant_items.get(user, set()))  # Intersection count
            if k > 0:  # Avoid division by zero
                precisions[user] = round(num_relevant_at_k / min(len(recommended_items), k), 3)  # Compute Precision@k

    return precisions



def mean_average_precision(top_k: Dict[str, List[str]], df_test: pd.DataFrame, k: int) -> float:
    """Compute mean average precision (MAP@k)
    Args:
        top_k: A dictionary where keys are user ids (str) and values are lists of (item_id, rating_estimation) tuples.
        df_test: Pandas DataFrame containing user-item ratings in the test split.
        k: The number of recommendations to output for each user.
    Returns:
        MAP@k (float)
    """
    
    average_precision_users = []
    
    # Get relevant items per user
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()

    for user, recommended_items in top_k.items():
        relevant_set = relevant_items.get(user, set())  # Get relevant items, default to empty set
        
        num_relevant = 0
        precision_sum = 0.0
        
        for i, (item, _) in enumerate(recommended_items[:k]):  # Iterate over top-K items
            if item in relevant_set:
                num_relevant += 1
                precision_sum += num_relevant / (i + 1)  # Precision at each relevant item

        # Avoid division by zero
        avg_precision = precision_sum / min(len(recommended_items), k) if num_relevant > 0 else 0
        average_precision_users.append(avg_precision)

    return np.mean(average_precision_users) if average_precision_users else 0.0


def mean_reciprocal_rank(top_k: Dict[str, List[str]], df_test: pd.DataFrame, k: int) -> float:
    """Compute mean reciprocal rank (MRR@k)
    Args:
        top_k: A dictionary where keys are user ids (str) and values are lists of (item_id, rating_estimation) tuples.
        df_test: Pandas DataFrame containing user-item ratings in the test split.
        k: The number of recommendations to output for each user.
    Returns:
        MRR@k (float)
    """
    
    reciprocal_ranks = []
    
    # Get relevant items per user
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()

    for user, recommended_items in top_k.items():
        relevant_set = relevant_items.get(user, set())  # Get relevant items, default to empty set
        found_relevant = False
        
        for i, (item, _) in enumerate(recommended_items[:k]):  # Iterate over top-K items
            if item in relevant_set:  # Find first relevant item
                reciprocal_ranks.append(1 / (i + 1))
                found_relevant = True
                break  # Stop after first relevant item

        if not found_relevant:
            reciprocal_ranks.append(0)  # Assign 0 if no relevant item is found

    return np.mean(reciprocal_ranks) if reciprocal_ranks else 0.0



def hit_rate(top_k: Dict[str, List[str]],
             df_test: pd.DataFrame) -> float:
    """Compute the hit rate
    Args:
        top_k: A dictionary where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n (output of get_top_k())
        df_test: Pandas DataFrame containing user-item ratings in 
            the test split.
    Returns:
        The average hit rate
    """

    hits = 0
    # Get relevant items per user
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()
    total_users = len(df_test[df_test['new_label'] == 1]['user_id'].unique())

    for user, recommended_items in top_k.items():
        recommended_set = {item for item, _ in recommended_items}  # Extract recommended item IDs
        if user in relevant_items:
            if recommended_set & relevant_items[user]:  # Check if there is any intersection
                hits += 1  

    return round(hits / total_users, 3) if total_users > 0 else 0.0



def coverage(top_k: Dict[str, List[str]], df_test: pd.DataFrame, df_train: pd.DataFrame) -> float:
    """
    Compute catalog coverage.

    Args:
        top_k: A dictionary where keys are user (raw) ids and values are lists of tuples:
               [(raw item id, rating estimation), ...] (output of get_top_k()).
        df_test: Pandas DataFrame containing the training data (user-item interactions).

    Returns:
        Coverage as a float (rounded to 3 decimals).
    """
    if not top_k:
        return 0.0  # No recommendations made

    recommended_items = {item for recommendations in top_k.values() for item, _ in recommendations}
    all_items = set(df_train["item_id"].unique()) | set(df_test["item_id"].unique()) 

    coverage_score = len(recommended_items) / len(all_items) if all_items else 0

    return round(coverage_score, 3)  # Round to 3 decimal places

In [ ]:
print("Metrics for Parallel Strategy:")
# PRECISION
precisions_nb = precision_at_k(top10_parallel, test_df, k=10)
print("Averaged P@10: {:.3f}".format(sum(prec for prec in precisions_nb.values()) / len(precisions_nb)))
# MAP 
map_nb = mean_average_precision(top10_parallel, test_df, k=10)
print("MAP@10: {:.3f}".format(map_nb))
# MRR
mrr_nb = mean_reciprocal_rank(top10_parallel, test_df, k=10)
print("MRR@10: {:.3f}".format(mrr_nb))
# hit rate
hit_r = hit_rate(top10_parallel, test_df)
print("Hit rate@10: {:.3f}".format(hit_r))
# coverage
cover = coverage(top10_parallel, test_df, train_df)
print("Coverage@10: {:.3f}".format(cover))

In [ ]:
print("Metrics for Switch Strategy:")
# PRECISION
precisions_nb = precision_at_k(top10_switch, test_df, k=10)
print("Averaged P@10: {:.3f}".format(sum(prec for prec in precisions_nb.values()) / len(precisions_nb)))
# MAP 
map_nb = mean_average_precision(top10_switch, test_df, k=10)
print("MAP@10: {:.3f}".format(map_nb))
# MRR
mrr_nb = mean_reciprocal_rank(top10_switch, test_df, k=10)
print("MRR@10: {:.3f}".format(mrr_nb))
# hit rate
hit_r = hit_rate(top10_switch, test_df)
print("Hit rate@10: {:.3f}".format(hit_r))
# coverage
cover = coverage(top10_switch, test_df, train_df)
print("Coverage@10: {:.3f}".format(cover))

In [ ]:
print("Metrics for Pipeline Strategy:")
# PRECISION
precisions_nb = precision_at_k(top10_pipeline, test_df, k=10)
print("Averaged P@10: {:.3f}".format(sum(prec for prec in precisions_nb.values()) / len(precisions_nb)))
# MAP 
map_nb = mean_average_precision(top10_pipeline, test_df, k=10)
print("MAP@10: {:.3f}".format(map_nb))
# MRR
mrr_nb = mean_reciprocal_rank(top10_pipeline, test_df, k=10)
print("MRR@10: {:.3f}".format(mrr_nb))
# hit rate
hit_r = hit_rate(top10_pipeline, test_df)
print("Hit rate@10: {:.3f}".format(hit_r))
# coverage
cover = coverage(top10_pipeline, test_df, train_df)
print("Coverage@10: {:.3f}".format(cover))

### long tail analysis

In [ ]:
from typing import Dict, List, Tuple
import pandas as pd

def hit_rate(top_k: Dict[str, List[tuple]], df_test: pd.DataFrame) -> float:
    hits = 0
    relevant_items = df_test[df_test['new_label'] == 1].groupby("user_id")["item_id"].apply(set).to_dict()
    total_users = len(relevant_items)

    for user, recommended_items in top_k.items():
        if user in relevant_items:
            recommended_set = {item for item, _ in recommended_items}
            if recommended_set & relevant_items[user]:
                hits += 1

    return round(hits / total_users, 3) if total_users > 0 else 0.0

def top_and_last_20_hit_rate(top_k: Dict[str, List[tuple]], df_test: pd.DataFrame, df_train: pd.DataFrame) -> Tuple[float, float]:
    user_interaction_counts = df_train.groupby('user_id').size().sort_values(ascending=False)
    num_users = len(user_interaction_counts)
    top_20_users = set(user_interaction_counts.head(int(0.2 * num_users)).index)
    last_20_users = set(user_interaction_counts.tail(int(0.2 * num_users)).index)

    df_test_top_20 = df_test[df_test['user_id'].isin(top_20_users)]
    df_test_last_20 = df_test[df_test['user_id'].isin(last_20_users)]

    top_20_hr = hit_rate(top_k, df_test_top_20)
    last_20_hr = hit_rate(top_k, df_test_last_20)

    return top_20_hr, last_20_hr

top20_hit_pc, last20_hit_pc = top_and_last_20_hit_rate(top10_parallel, test_df, train_df)
top20_hit_sw, last20_hit_sw = top_and_last_20_hit_rate(top10_switch, test_df, train_df)
top20_hit_pp, last20_hit_pp = top_and_last_20_hit_rate(top10_pipeline, test_df, train_df)

print("Parallel combination Hit Rate - Top 20%: {:.3f}, Last 20%: {:.3f}".format(top20_hit_pc, last20_hit_pc))
print("Switching Hit Rate - Top 20%: {:.3f}, Last 20%: {:.3f}".format(top20_hit_sw, last20_hit_sw))
print("Pipelining Hit Rate - Top 20%: {:.3f}, Last 20%: {:.3f}".format(top20_hit_pp, last20_hit_pp))



In [ ]:
from typing import Dict, List, Set, Tuple
def coverage(top_k: Dict[str, List[str]], relevant_items: Set[str]) -> float:

    recommended_items = {item for recs in top_k.values() for item, _ in recs}
    matched = recommended_items & relevant_items
    return round(len(matched)/len(relevant_items), 3) if relevant_items else 0

def get_item_groups(df_train: pd.DataFrame) -> Tuple[Set[str], Set[str]]:

    item_counts = df_train['item_id'].value_counts()
    split_idx = int(len(item_counts) * 0.2)
    return set(item_counts.head(split_idx).index), set(item_counts.tail(split_idx).index)

# Correct usage
top_items, tail_items = get_item_groups(train_df)  # Use actual training data

# Calculate coverage for different groups
def calculate_group_coverage(top_k: Dict[str, List[str]], items: Set[str]) -> float:
    return coverage(top_k, items)

top20_cov_pc = calculate_group_coverage(top10_parallel, top_items)
last20_cov_pc = calculate_group_coverage(top10_parallel, tail_items)

print(f"Parallel  coverage - Top 20%: {top20_cov_pc}, Tail 20%: {last20_cov_pc}")

top20_cov_sw = calculate_group_coverage(top10_switch, top_items)
last20_cov_sw = calculate_group_coverage(top10_switch, tail_items)

print(f"Switching  coverage - Top 20%: {top20_cov_sw}, Tail 20%: {last20_cov_sw}")

top20_cov_pp = calculate_group_coverage(top10_pipeline, top_items)
last20_cov_tpp = calculate_group_coverage(top10_pipeline, tail_items)

print(f"Pipelining  coverage - Top 20%: {top20_cov_pp}, Tail 20%: {last20_cov_tpp}")

Plots for all

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams.update({
    'axes.titlesize': 20,
    'axes.labelsize': 18, 
    'xtick.labelsize': 14, 
    'ytick.labelsize': 14,  
    'legend.fontsize': 15,  
})

models = ['KNN', 'SVD','TF-IDF', 'Parallel', 'Switching', 'Pipelining', 'TopPop']
top_20_hr = [0.050, 0.100, 0.167, 0.117, 0.167, 0.150, 0.033] 
last_20_hr = [0.181, 0.181, 0.163, 0.169, 0.163, 0.119, 0.069] 

x = np.arange(len(models))
width = 0.35 


fig, ax = plt.subplots(figsize=(10, 6))

rects1 = ax.bar(x - width/2, top_20_hr, width, label='Top 20% Hit Rate', color='skyblue')
rects2 = ax.bar(x + width/2, last_20_hr, width, label='Last 20% Hit Rate', color='red')

ax.set_xlabel('Recommender Systems')
ax.set_ylabel('Hit Rate')
ax.set_title('Hit Rate Comparison: Top 20% vs Last 20% Users')
ax.set_xticks(x)
ax.set_xticklabels(models,ha='right')
ax.legend()

def add_labels(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(rects1)
add_labels(rects2)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({
    'axes.titlesize': 20, 
    'axes.labelsize': 18,  
    'xtick.labelsize': 14,
    'ytick.labelsize': 14, 
    'legend.fontsize': 15,
})

models = ['KNN', 'SVD', 'TF-IDF', 'Parallel', 'Switching', 'Pipelining', 'TopPop']
top_20_cov = [0.99, 0.277, 0.782, 0.683, 0.782, 0.307, 0.01] 
tail_20_cov = [0.564, 0.188, 0.564, 0.446, 0.564, 0.238, 0.0] 

x = np.arange(len(models))
width = 0.35 

fig, ax = plt.subplots(figsize=(10, 6))

rects1 = ax.bar(x - width/2, top_20_cov, width, label='Top 20% Coverage', color='green')
rects2 = ax.bar(x + width/2, tail_20_cov, width, label='Tail 20% Coverage', color='orange')

ax.set_xlabel('Recommender Systems')
ax.set_ylabel('Coverage')
ax.set_title('Coverage Comparison: Top 20% vs Tail 20% Items')
ax.set_xticks(x)
ax.set_xticklabels(models, ha='right')
ax.legend()

def add_labels(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(rects1)
add_labels(rects2)

plt.tight_layout()
plt.show()